# Практическое задание к уроку 8. Рекуррентные нейронные сети RNN LSTM GRU


Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи 

1. построить свёрточные архитектуры 

2. построить различные архитектуры с RNN

3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install daal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install typing-extensions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install dca

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [8]:
import dca
import pkg_resources

import pandas as pd
import numpy as np
import re

pd.set_option('max_colwidth', 120)

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, SpatialDropout1D, concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import SimpleRNN, GRU, Masking
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.preprocessing.text import Tokenizer

#### Обработка данных

In [9]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")
df_val = pd.read_csv("/content/val.csv")

In [10]:
df_train.head(5)

,id,text,class
0,0,"@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.",0
1,1,"RT @GalyginVadim: Ребята и девчата!\nВсе в кино!!! ""Вот Это Любовь!""\nСегодня! Завтра! И потом!)))))\n#вотэтолюбовь",1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Запретный плод. :),1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса не боится мороза и .......)),1


In [11]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [12]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [13]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [14]:
y_train = df_train['class'].values
y_val = df_val['class'].values

#### Сверточная сеть

In [15]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    output_dim=128, 
                    input_length=training_length, 
                    trainable=True,
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [16]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 164s 509ms/step - loss: 0.5961 - accuracy: 0.6682 - val_loss: 0.4793 - val_accuracy: 0.7600
Epoch 2/10
319/319 [==============================] - 157s 494ms/step - loss: 0.2633 - accuracy: 0.8962 - val_loss: 0.5464 - val_accuracy: 0.7466


In [17]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 35ms/step - loss: 0.6106 - accuracy: 0.7334


Test score: 0.6105940937995911
Test accuracy: 0.7333685755729675


In [18]:
compare_result = pd.DataFrame(columns=['Model', 'Score', 'Accuracy'])
compare_result.loc[0] = ['CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369


#### RNN

Простая рекуррентная сеть

In [19]:
tf.__version__

'2.4.4'

In [20]:
keras.__version__

'2.4.0'

In [21]:
np.__version__

'1.18.5'

In [22]:
!pip install numpy==1.18.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.18.5 which is incompatible.
tensorflow 2.4.4 requires numpy~=1.19.2, but you have numpy 1.18.5 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.18.5 which is incompatible.
jaxlib 0.3.14+cuda11.cudnn805 requires numpy>=1.19, but you have numpy 1.18.5 which is incompatible.
jax 0.3.14 requires numpy>=1.19, but you have numpy 1.18.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you h

In [23]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [24]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 81s 246ms/step - loss: 0.6174 - accuracy: 0.6365 - val_loss: 0.4979 - val_accuracy: 0.7547
Epoch 2/10
319/319 [==============================] - 54s 169ms/step - loss: 0.2917 - accuracy: 0.8842 - val_loss: 0.5566 - val_accuracy: 0.7452


In [25]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 25ms/step - loss: 0.5978 - accuracy: 0.7345


Test score: 0.597795307636261
Test accuracy: 0.7344707250595093


In [26]:
compare_result.loc[1] = ['RNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471


#### LSTM

In [27]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [28]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 134s 410ms/step - loss: 0.6089 - accuracy: 0.6517 - val_loss: 0.4951 - val_accuracy: 0.7517
Epoch 2/10
319/319 [==============================] - 132s 414ms/step - loss: 0.3284 - accuracy: 0.8664 - val_loss: 0.5303 - val_accuracy: 0.7418


In [29]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 54ms/step - loss: 0.5527 - accuracy: 0.7368


Test score: 0.5526803135871887
Test accuracy: 0.7368072867393494


In [30]:
compare_result.loc[2] = ['LSTM', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471
2,LSTM,0.552680,0.736807


#### GRU

In [31]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True,
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [32]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train,batch_size=512,epochs=10,verbose=1,validation_split=0.1,callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 94s 286ms/step - loss: 0.6092 - accuracy: 0.6506 - val_loss: 0.4940 - val_accuracy: 0.7548
Epoch 2/10
319/319 [==============================] - 90s 283ms/step - loss: 0.3185 - accuracy: 0.8697 - val_loss: 0.5296 - val_accuracy: 0.7479


In [33]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 44ms/step - loss: 0.5728 - accuracy: 0.7372


Test score: 0.5728344917297363
Test accuracy: 0.7372481822967529


In [34]:
compare_result.loc[3] = ['GRU', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471
2,LSTM,0.552680,0.736807
3,GRU,0.572834,0.737248


In [35]:
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471
2,LSTM,0.552680,0.736807
3,GRU,0.572834,0.737248


#### Совместные архитектуры CNN -> RNN

##### CNN-SimpleRNN

In [36]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 65s 201ms/step - loss: 0.6154 - accuracy: 0.6386 - val_loss: 0.4901 - val_accuracy: 0.7589
Epoch 2/10
319/319 [==============================] - 61s 192ms/step - loss: 0.2910 - accuracy: 0.8832 - val_loss: 0.5632 - val_accuracy: 0.7389


In [37]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 31ms/step - loss: 0.5861 - accuracy: 0.7338


Test score: 0.5860626697540283
Test accuracy: 0.7338094711303711


In [38]:
compare_result.loc[4] = ['CNN-SimpleRNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471
2,LSTM,0.552680,0.736807
3,GRU,0.572834,0.737248
4,CNN-SimpleRNN,0.586063,0.733809


##### CNN - LSTM

In [39]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 120s 367ms/step - loss: 0.6062 - accuracy: 0.6525 - val_loss: 0.4873 - val_accuracy: 0.7589
Epoch 2/10
319/319 [==============================] - 117s 367ms/step - loss: 0.2991 - accuracy: 0.8783 - val_loss: 0.5454 - val_accuracy: 0.7452


In [40]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 67ms/step - loss: 0.5908 - accuracy: 0.7364


Test score: 0.5908342003822327
Test accuracy: 0.7363664507865906


In [41]:
compare_result.loc[5] = ['CNN - LSTM', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471
2,LSTM,0.552680,0.736807
3,GRU,0.572834,0.737248
4,CNN-SimpleRNN,0.586063,0.733809
5,CNN - LSTM,0.590834,0.736366


##### CNN-GRU

In [42]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 112s 341ms/step - loss: 0.6025 - accuracy: 0.6546 - val_loss: 0.4864 - val_accuracy: 0.7602
Epoch 2/10
319/319 [==============================] - 108s 338ms/step - loss: 0.2958 - accuracy: 0.8796 - val_loss: 0.5566 - val_accuracy: 0.7440


In [43]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 57ms/step - loss: 0.6081 - accuracy: 0.7329


Test score: 0.60807865858078
Test accuracy: 0.732883632183075


In [44]:
compare_result.loc[6] = ['CNN-GRU', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471
2,LSTM,0.552680,0.736807
3,GRU,0.572834,0.737248
4,CNN-SimpleRNN,0.586063,0.733809
5,CNN - LSTM,0.590834,0.736366
6,CNN-GRU,0.608079,0.732884


#### Совместные архитектуры RNN -> CNN

##### SimpleRNN-CNN

In [45]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Conv1D(64, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 74s 225ms/step - loss: 0.6825 - accuracy: 0.5291 - val_loss: 0.6594 - val_accuracy: 0.5607
Epoch 2/10
319/319 [==============================] - 70s 221ms/step - loss: 0.6197 - accuracy: 0.5922 - val_loss: 0.6701 - val_accuracy: 0.5606


In [46]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 36ms/step - loss: 0.6782 - accuracy: 0.5509


Test score: 0.6782004833221436
Test accuracy: 0.5508759617805481


In [47]:
compare_result.loc[7] = ['SimpleRNN-CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471
2,LSTM,0.552680,0.736807
3,GRU,0.572834,0.737248
4,CNN-SimpleRNN,0.586063,0.733809
5,CNN - LSTM,0.590834,0.736366
6,CNN-GRU,0.608079,0.732884
7,SimpleRNN-CNN,0.678200,0.550876


##### LSTM - CNN

In [48]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30)
          )
model.add(LSTM(64, recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 121s 368ms/step - loss: 0.6836 - accuracy: 0.5268 - val_loss: 0.6628 - val_accuracy: 0.5555
Epoch 2/10
319/319 [==============================] - 117s 367ms/step - loss: 0.6278 - accuracy: 0.5870 - val_loss: 0.6694 - val_accuracy: 0.5609


In [49]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 63ms/step - loss: 0.6735 - accuracy: 0.5511


Test score: 0.6735475659370422
Test accuracy: 0.5510788559913635


In [50]:
compare_result.loc[8] = ['LSTM - CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471
2,LSTM,0.552680,0.736807
3,GRU,0.572834,0.737248
4,CNN-SimpleRNN,0.586063,0.733809
5,CNN - LSTM,0.590834,0.736366
6,CNN-GRU,0.608079,0.732884
7,SimpleRNN-CNN,0.678200,0.550876
8,LSTM - CNN,0.673548,0.551079


##### GRU-CNN

In [51]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 118s 359ms/step - loss: 0.6817 - accuracy: 0.5284 - val_loss: 0.6613 - val_accuracy: 0.5549
Epoch 2/10
319/319 [==============================] - 113s 356ms/step - loss: 0.6260 - accuracy: 0.5844 - val_loss: 0.6689 - val_accuracy: 0.5541


In [52]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 65ms/step - loss: 0.6720 - accuracy: 0.5458


Test score: 0.6720185279846191
Test accuracy: 0.5457831621170044


In [53]:
compare_result.loc[9] = ['GRU - CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.610594,0.733369
1,RNN,0.597795,0.734471
2,LSTM,0.552680,0.736807
3,GRU,0.572834,0.737248
4,CNN-SimpleRNN,0.586063,0.733809
5,CNN - LSTM,0.590834,0.736366
6,CNN-GRU,0.608079,0.732884
7,SimpleRNN-CNN,0.678200,0.550876
8,LSTM - CNN,0.673548,0.551079
9,GRU - CNN,0.672019,0.545783


***Вывод:*** кроме RNN -> CNN модели показывают примерно одинаковую результативность.